- Generate epoch files with epoch length as 10 seconds.
- Load the epoch files and label files with 10 second worth of labels.
- Train the model.
- Use the model to make predictions on test set and compare with actual annotations we have.
- Make the confusion matrix.
- Do it for: original model and custom features model.
- Use those models for 22 random participants.

In [1]:
def get_files(data_path) -> None:
    total_csv_zipped = []
    for path, dirnames, filenames in os.walk(data_path):
        for file in filenames:
            if file.endswith(".csv.gz"):
                total_csv_zipped.append(os.path.join(path, file))
                
            if file.endswith(".csv") and file[0]!='c': #ignore the capture24 file
                # paths contains the base directory for that file.
                # dirnames contains other directories within this folder.
                # filenames contains the list of filenames within path.
                total_csv_zipped.append(os.path.join(path, file))
    return sorted(total_csv_zipped)

In [2]:
import pandas as pd
import os
epoch_size = 1000 # 10 seconds

epochs = "/home/aayush/accelerometer/compare_classification/epoch_data/10_sec/with_extra_features/epoch_data"
epoch_files = get_files(epochs)

labels = "/home/yacine/accel/capture24/participants/"
label_files = get_files(labels)

In [3]:
import csv
def create_labels_dict(labels_dict_location = "/home/aayush/accelerometer/accprocess/anno-label.csv"):
    labels_dict = {}
    with open(labels_dict_location, "r") as annotation_dict:
        reader = csv.DictReader(annotation_dict)
        for row in reader:
            if labels_dict.get(row['annotation']) is None:
                labels_dict[row['annotation']] = [row['label:Walmsley2020']]
            else:
                labels_dict[row['annotation']].append(row['label:Walmsley2020'])
    return labels_dict

labels_dict = create_labels_dict()

# replace the annotated labels with the same format of strings as predicted labels using mapping from labels_dict
flat_dict = {k: v[0] for k, v in labels_dict.items()}

In [4]:
def parse_datetime(dt_string):
    clean_datetime_str = dt_string.split('[')[0].strip()
    dt_object = pd.to_datetime(clean_datetime_str)
    return dt_object


def parse_datetime_df_time(dt_string):
    clean_datetime_str = dt_string.split('[')[0].strip()
    clean_datetime_str = clean_datetime_str.split('+')[0].strip()
    dt_object = pd.to_datetime(clean_datetime_str)
    return dt_object

In [5]:
all_epoch_dfs = []
for i, (epochfilename, labelfilename) in enumerate(zip(epoch_files, label_files)):
    epoch_df = pd.read_csv(epochfilename)
    label_df = pd.read_csv(labelfilename)

    # Take the timestamp after every thirty seconds
    label_df = label_df[["annotation", "time"]][0::epoch_size]
    
    # Convert to datetime object
    label_df["time"] = label_df["time"].apply(parse_datetime)

    # replace the label with the categories we expect
    label_df['annotation'] = label_df['annotation'].replace(flat_dict)

    # convert epoch df time stamp with datetime object
    epoch_df['time'] = epoch_df['time'].apply(parse_datetime_df_time)
    epoch_df["participant"] = float(epochfilename.split("/")[-1].split("-")[0][1:])

    # predropped_epoch = len(epoch_df)
    # predropped_label = len(label_df)
    
    # # keep only those time stamps which are present in both epoch and label df
    # label_df = label_df[label_df["time"].isin(epoch_df['time'])]
    # epoch_df = epoch_df[epoch_df["time"].isin(label_df['time'])]

    # print(f"We removed {predropped_label-len(label_df)} rows from label df because timestamps were not in epoch df.")
    # print(f"We removed {predropped_epoch-len(epoch_df)} rows from epoch_df because timestamps were not in label_df.")
    
    label_df = label_df[["time", "annotation"]]
    label_df.set_index("time", inplace=True)
    epoch_df.set_index("time", inplace=True)
    
    # epoch_df["label"] = label_df['annotation']
    epoch_df=epoch_df.join(label_df, on="time")
    
    all_epoch_dfs.append(epoch_df)
    print(f"{i+1}/{len(epoch_files)} done...")

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3442: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


1/151 done...
2/151 done...
3/151 done...
4/151 done...
5/151 done...
6/151 done...
7/151 done...
8/151 done...
9/151 done...
10/151 done...
11/151 done...
12/151 done...
13/151 done...
14/151 done...
15/151 done...
16/151 done...
17/151 done...
18/151 done...
19/151 done...
20/151 done...
21/151 done...
22/151 done...
23/151 done...
24/151 done...
25/151 done...
26/151 done...
27/151 done...
28/151 done...
29/151 done...
30/151 done...
31/151 done...
32/151 done...
33/151 done...
34/151 done...
35/151 done...
36/151 done...
37/151 done...
38/151 done...
39/151 done...
40/151 done...
41/151 done...
42/151 done...
43/151 done...
44/151 done...
45/151 done...
46/151 done...
47/151 done...
48/151 done...
49/151 done...
50/151 done...
51/151 done...
52/151 done...
53/151 done...
54/151 done...
55/151 done...
56/151 done...
57/151 done...
58/151 done...
59/151 done...
60/151 done...
61/151 done...
62/151 done...
63/151 done...
64/151 done...
65/151 done...
66/151 done...
67/151 done...
68/1

In [7]:
epochs = pd.concat(all_epoch_dfs)

In [8]:
df = epochs
df.head()

,enmoTrunc,enmoAbs,xMean,yMean,zMean,xRange,yRange,zRange,xStd,yStd,...,p625,totalPower,temp,samples,dataErrors,clipsBeforeCalibr,clipsAfterCalibr,rawSamples,participant,annotation
time,,,,,,,,,,,,,,,,,,,,,
2016-11-13 02:18:00,0.000843,0.003434,-0.460820,-0.523185,0.714398,0.015839,0.031264,0.031827,0.005733,0.006868,...,-17.894722,-14.462759,0.0,1000,0,0,0,1001,1.0,sleep
2016-11-13 02:18:10,0.000818,0.002860,-0.461200,-0.523639,0.714542,0.015839,0.031264,0.031827,0.006072,0.007069,...,-17.949640,-14.751847,0.0,1000,0,0,0,1002,1.0,sleep
2016-11-13 02:18:20,0.000639,0.002645,-0.463512,-0.520434,0.715178,0.015839,0.031264,0.031827,0.007419,0.004226,...,-18.121028,-15.372644,0.0,1000,0,0,0,1002,1.0,sleep
2016-11-13 02:18:30,0.000597,0.002615,-0.463957,-0.519730,0.715366,0.015904,0.031264,0.031827,0.007570,0.003598,...,-18.189799,-15.489662,0.0,1000,0,0,0,1002,1.0,sleep
2016-11-13 02:18:40,0.000793,0.002516,-0.465270,-0.519528,0.715324,0.015968,0.031282,0.032034,0.007853,0.004035,...,-17.755043,-14.841447,0.0,1000,0,0,0,1002,1.0,sleep


In [9]:
df.rename(columns={'annotation':'label'}, inplace=True)
print(len(df))

1398036


In [10]:
print(df['label'].unique())

['sleep' nan 'light' 'sedentary' 'moderate-vigorous']


In [11]:
# df = df.drop(labels=["time"], axis=1)

In [12]:
df = df.dropna(subset=["label"])
print(df['label'].unique())

['sleep' 'light' 'sedentary' 'moderate-vigorous']


In [13]:
# df["label"] = df["label"].astype('string', copy=False)
len(df)

883480

In [14]:
df.to_csv("/home/aayush/accelerometer/compare_classification/training_data/10_sec/extra_features/10s_training_with_annotation_labels.csv", index=False)

In [15]:
test_participants = "101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151"
test_participants = [float(t) for t in test_participants.split(",")]
    
test_features_df = df[df["participant"].isin(test_participants)].iloc[:, :-2]
test_labels_df = df[df["participant"].isin(test_participants)]["label"]

print(len(test_features_df))
print(test_labels_df.unique())

291961
['sleep' 'sedentary' 'light' 'moderate-vigorous']


In [16]:
train_all_df = df[~df["participant"].isin(test_participants)]
train_features_df = df[~df["participant"].isin(test_participants)].iloc[:, :-2]
train_labels_predict = df[~df["participant"].isin(test_participants)]["label"]

print(len(train_features_df))
print(train_labels_predict)

591519
time
2016-11-13 02:18:00    sleep
2016-11-13 02:18:10    sleep
2016-11-13 02:18:20    sleep
2016-11-13 02:18:30    sleep
2016-11-13 02:18:40    sleep
                       ...  
2016-05-31 02:13:20    sleep
2016-05-31 02:13:30    sleep
2016-05-31 02:13:40    sleep
2016-05-31 02:13:50    sleep
2016-05-31 02:14:00    sleep
Name: label, Length: 591519, dtype: object


In [17]:
def _Model(**kwargs):
    return BalancedRandomForestClassifier(
        n_estimators=3000,
        max_depth=10,
        min_samples_leaf=1,
        replacement=True,
        sampling_strategy='not minority',
        random_state=42,
        **kwargs
    )

In [18]:
from imblearn.ensemble import BalancedRandomForestClassifier

print('Training...')
model = _Model(n_jobs=10, verbose=1)
# fit the model as numpy array so that we do not get warnings during prediction
X_features = train_features_df.to_numpy()
X_labels = train_labels_predict.to_numpy()
model = model.fit(X_features, X_labels)
model.verbose = 0  # silence future calls to .predict()
labels = model.classes_

Training...


[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    6.5s
[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:   32.5s
[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed:  1.3min
[Parallel(n_jobs=10)]: Done 780 tasks      | elapsed:  2.3min
[Parallel(n_jobs=10)]: Done 1230 tasks      | elapsed:  3.6min
[Parallel(n_jobs=10)]: Done 1780 tasks      | elapsed:  5.4min
[Parallel(n_jobs=10)]: Done 2430 tasks      | elapsed:  7.2min
[Parallel(n_jobs=10)]: Done 3000 out of 3000 | elapsed:  8.8min finished


In [19]:
print(labels)
pid = df[~df["participant"].isin(test_participants)]["participant"].to_numpy()
print(len(pid))
print(len(train_labels_predict))

['light' 'moderate-vigorous' 'sedentary' 'sleep']
591519
591519


In [20]:
def saveToTar(tarOut, **kwargs):
    """Save objects to tar file. Objects must be passed as keyworded arguments,
    then the key is used for the object name in the tar file.

    :param **kwargs: Objects to be saved passed as keyworded arguments.

    :return: tar file written to <tarOut>
    :rtype: void
    """

    try:

        tmpdir = tempfile.mkdtemp()

        with tarfile.open(tarOut, mode='w') as tf:

            for key, val in kwargs.items():
                pth = os.path.join(tmpdir, key)
                joblib.dump(val, pth, compress=True)
                tf.add(pth, arcname=key)

        print('Models saved to', tarOut)

    finally:

        try:
            shutil.rmtree(tmpdir)
        except OSError as e:
            print("Error: %s - %s." % (e.filename, e.strerror))


def getFileFromTar(tarArchive, targetFile):
    """Read file from tar

    This is currently more tricky than it should be see
    https://github.com/numpy/numpy/issues/7989

    :param str tarArchive: Input tarfile object
    :param str targetFile: Target individual file within .tar

    :return: file object byte stream
    :rtype: object
    """

    with tarfile.open(tarArchive, 'r') as t:
        b = BytesIO()
        try:
            b.write(t.extractfile(targetFile).read())
        except KeyError:
            return None
        b.seek(0)

    return b


def trainHMM(Y_prob, Y_true, labels=None, uniform_prior=True):
    """ https://en.wikipedia.org/wiki/Hidden_Markov_model

    :return: Dictionary containing prior, emission and transition
        matrices, and corresponding labels.
    :rtype: dict

    """

    if labels is None:
        labels = np.unique(Y_true)

    if uniform_prior:
        # All labels with equal probability
        prior = np.ones(len(labels)) / len(labels)
    else:
        # Label probability equals empirical rate
        prior = np.mean(Y_true.reshape(-1, 1) == labels, axis=0)

    emission = np.vstack(
        [np.mean(Y_prob[Y_true == label], axis=0) for label in labels]
    )
    transition = np.vstack(
        [np.mean(Y_true[1:][(Y_true == label)[:-1]].reshape(-1, 1) == labels, axis=0)
            for label in labels]
    )

    params = {'prior': prior, 'emission': emission, 'transition': transition, 'labels': labels}

    return params

In [21]:
from sklearn.model_selection import cross_val_predict, cross_val_score
import numpy as np

print('Cross-predicting to derive the observations for HMM...')

NJOBS_PER_CV_MODEL = min(2, 10)
cvp = cross_val_predict(
    _Model(n_jobs=NJOBS_PER_CV_MODEL), X_features, X_labels, groups=pid,
    cv=10,
    n_jobs=10 // NJOBS_PER_CV_MODEL,
    method="predict_proba",
    verbose=3,
)

Cross-predicting to derive the observations for HMM...


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done   5 out of  10 | elapsed: 24.2min remaining: 24.2min
[Parallel(n_jobs=5)]: Done  10 out of  10 | elapsed: 42.0min finished


In [22]:
labels

array(['light', 'moderate-vigorous', 'sedentary', 'sleep'], dtype=object)

In [23]:
print('Training HMM...')
# train_labels_predict_array = np.array(train_labels_predict)
# hmmParams = trainHMM(cvp,  train_labels_predict_array)

hmmParams = trainHMM(cvp,  X_labels)

Training HMM...


In [24]:
hmmParams

{'prior': array([0.25, 0.25, 0.25, 0.25]),
 'emission': array([[0.33993685, 0.28638068, 0.291851  , 0.08183147],
        [0.30721581, 0.36509814, 0.26417025, 0.0635158 ],
        [0.2908913 , 0.25157763, 0.35140236, 0.10612872],
        [0.08167573, 0.06034193, 0.10836656, 0.74961579]]),
 'transition': array([[9.82898529e-01, 3.09350273e-03, 1.35765298e-02, 4.31438066e-04],
        [1.25358707e-02, 9.85160852e-01, 2.15224286e-03, 1.51034587e-04],
        [5.79335573e-03, 2.06763554e-04, 9.93805046e-01, 1.94834887e-04],
        [3.61679562e-04, 1.95502466e-05, 1.07526356e-04, 9.99511244e-01]]),
 'labels': array(['light', 'moderate-vigorous', 'sedentary', 'sleep'], dtype=object)}

In [25]:
# METs = {y: train_all_df[train_all_df["label"] == y].groupby("label")["MET"].mean().mean() 
#     for y in model.classes_}
METs = {'light': 2.270833333333333,
 'moderate-vigorous': 4.682608695652173,
 'sedentary': 1.5634920634920637,
 'sleep': 0.9499999999999998}
METs

{'light': 2.270833333333333,
 'moderate-vigorous': 4.682608695652173,
 'sedentary': 1.5634920634920637,
 'sleep': 0.9499999999999998}

In [26]:
import os
import tempfile
import shutil
import joblib
import tarfile

featureCols = np.array(train_features_df.columns)
# Write model to file
outFile = os.path.join("/home/aayush/accelerometer/compare_classification/accProcess_output/10_sec/extra_features/model_used", '10s_extra_model.tar')
saveToTar(outFile,
          model=model,
          labels=labels,
          featureCols=featureCols,
          hmmParams=hmmParams,
          METs=METs)
print(f'Output trained model written to: {outFile}')

Models saved to /home/aayush/accelerometer/compare_classification/accProcess_output/10_sec/extra_features/model_used/10s_extra_model.tar
Output trained model written to: /home/aayush/accelerometer/compare_classification/accProcess_output/10_sec/extra_features/model_used/10s_extra_model.tar


In [28]:
# Display feature importances
feature_importances = pd.Series(model.feature_importances_, index=df.columns[:-2])
print(feature_importances.nlargest(30))  # Show the top 30 features

max          0.064296
xRange       0.059305
75thp        0.055442
sd           0.047031
minx         0.045065
zRange       0.043988
xStd         0.041275
mean         0.036784
fft2         0.031967
zStd         0.028829
fft1         0.027774
xMean        0.024979
yRange       0.023975
fft3         0.022431
maxx         0.022376
miny         0.017885
enmoAbs      0.016795
MAD          0.016660
minz         0.016036
enmoTrunc    0.014866
yawg         0.014275
fft4         0.014265
median       0.014052
MPD          0.013791
pmax         0.012818
fft5         0.012681
pitchg       0.012111
maxz         0.011601
yMean        0.011287
fft9         0.010703
dtype: float64


In [29]:
import sklearn.metrics as metrics

# test the performance
Y_labels_test_pred = model.predict(test_features_df)
print(metrics.classification_report(test_labels_df, Y_labels_test_pred))
testScore = metrics.f1_score(test_labels_df, Y_labels_test_pred, average='macro', zero_division=0)
print(f'Score: {testScore:.2f}')

/home/aayush/.local/lib/python3.10/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but BalancedRandomForestClassifier was fitted without feature names
  warnings.warn(


                   precision    recall  f1-score   support

            light       0.44      0.34      0.38     62361
moderate-vigorous       0.17      0.42      0.24     18008
        sedentary       0.67      0.55      0.60    113427
            sleep       0.86      0.92      0.89     98165

         accuracy                           0.62    291961
        macro avg       0.53      0.56      0.53    291961
     weighted avg       0.65      0.62      0.63    291961

Score: 0.53


In [30]:
def viterbi(Y_obs, hmm_params):
    """ Perform HMM smoothing over observations via Viteri algorithm

    https://en.wikipedia.org/wiki/Viterbi_algorithm

    :param dict hmm_params: Dictionary containing prior, emission and transition
        matrices, and corresponding labels

    :return: Smoothed sequence of activities
    :rtype: numpy.array
    """

    def log(x):
        SMALL_NUMBER = 1e-16
        return np.log(x + SMALL_NUMBER)

    prior = hmm_params['prior']
    emission = hmm_params['emission']
    transition = hmm_params['transition']
    labels = hmm_params['labels']

    nobs = len(Y_obs)
    nlabels = len(labels)

    Y_obs = np.where(Y_obs.reshape(-1, 1) == labels)[1]  # to numeric

    probs = np.zeros((nobs, nlabels))
    probs[0, :] = log(prior) + log(emission[:, Y_obs[0]])
    for j in range(1, nobs):
        for i in range(nlabels):
            probs[j, i] = np.max(
                log(emission[i, Y_obs[j]]) +
                log(transition[:, i]) +
                probs[j - 1, :])  # probs already in log scale
    viterbi_path = np.zeros_like(Y_obs)
    viterbi_path[-1] = np.argmax(probs[-1, :])
    for j in reversed(range(nobs - 1)):
        viterbi_path[j] = np.argmax(
            log(transition[:, viterbi_path[j + 1]]) +
            probs[j, :])  # probs already in log scale

    viterbi_path = labels[viterbi_path]  # to labels

    return viterbi_path

In [31]:
YpredHmm = viterbi(Y_labels_test_pred, hmmParams)

print('\nTest performance (HMM):')
print(metrics.classification_report(test_labels_df, YpredHmm))
testHmmScore = metrics.f1_score(test_labels_df, YpredHmm, average='macro', zero_division=0)
print(f'Score: {testHmmScore:.2f}')


Test performance (HMM):
                   precision    recall  f1-score   support

            light       0.51      0.33      0.40     62361
moderate-vigorous       0.18      0.44      0.25     18008
        sedentary       0.73      0.64      0.68    113427
            sleep       0.89      0.99      0.93     98165

         accuracy                           0.68    291961
        macro avg       0.58      0.60      0.57    291961
     weighted avg       0.70      0.68      0.68    291961

Score: 0.57
